In [20]:
# %load_ext autoreload
# %autoreload 2

import os,sys,warnings
import h5py, pandas
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit,fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 8
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
np.set_printoptions(legacy='1.25')

# mpl.rcParams.update({"axes.grid" : True})
import util as yu
yu.flag_fast=False

shift131=(2.9,0.5); shiftMpi0=(0,0); shiftMpiC=(-3.1,0.5)
doShift=True

#[label,sgm,flavor,a-correction,shift]
sgms={
    # lattice matrix element
    'this':['This work',(36.3,0.7),'2','a2',shift131], # mpi=131 O(a)-improved
    
    'this2':['This work',(34.4,1.3),'2','a2',shift131],
    'ETM24':['ETM24',(41.9,8.1),'211','a3',shiftMpiC], # Alexandrou:2024ozj mpi=140,137,141 O(a)-improved + a^2-extrapolation
    'ETM19':['ETM19',(41.6,3.8),'211','a2',shiftMpiC], # Alexandrou:2019brg mpi=140
    # 'ETM19_A2':['ETM19',()], # A2
    'Mainz23':['Mainz23',(43.7,3.6),'21','a2',shiftMpi0], # Agadjanov:2023efe mpi@134.8
    'Mainz23_wd':['Mainz23',(42.3,2.4),'21','a2',shiftMpi0], # Agadjanov:2023efe
    'Mainz23_prior':['Mainz23',(46.9,1.7),'21','a2',shiftMpi0], # Agadjanov:2023efe
    
    'PNDME21':['PNDME21',(41.9,4.9),'211','a2',shiftMpi0], # Gupta:2021ahb standard mpi=135
    'PNDME21_prior':['PNDME21',(59.6,7.4),'211','a2',shiftMpi0], # with prior from ChiPT
    
    'PNDME25':['PNDME25',(42,6),'211','a2',shiftMpi0], # Park:2025rxi standard mpi=135
    'PNDME25_prior':['PNDME25',(62,6),'211','a2',shiftMpi0], # with prior from ChiPT
    
    # lattice FH
    'BMW20':['BMW20',(37.4,3.0,4.1),'1111','a?',shiftMpi0], # Borsanyi:2020bpd mpi=134.8
    'RQCD23':['RQCD23',(43.9,4.7),'21','a3',shiftMpi0], # Bali:2023sdi mpi=134.8
    'QCDSF12':['QCDSF12',(37,8,6),'2','a2?',shiftMpiC], # Bali:2012qs mpi=138
    
    # phenomenology
    'Hofe23':['Hoferichter 23',(59.0,3.5),'phe','0',shiftMpiC], # Hoferichter:2023ptl; scattering length from pionic atom
    'Ruiz18':['Ruiz de Elvira 18',(58,5),'phe','0',shiftMpiC], # RuizdeElvira:2017stg; low-energy piN cross section
    'Frie19':['Friedman 19',(57,7),'phe','0',shiftMpiC], # Friedman:2019zhc; pionic atom with large Z used
}

fla2sty={'2':('r','s'),'21':('g','o'),'211':('b','d'),'1111':('b','d'),'phe':('purple','x')}

fig, axs = yu.getFigAxs(1,1,sharey=True)
ax=axs[0,0]

cases=['this','this2','ETM24','Mainz23_wd','PNDME25','Mainz23_prior','PNDME25_prior','RQCD23','BMW20','QCDSF12','Hofe23','Frie19','Ruiz18']

xlim=[10,95]
ax.set_xlim(xlim)
ax.set_xticks([20,40,60,80])
ax.set_xlabel(r'$\sigma_{\pi N}$ [MeV]',fontsize=20)
ylim=[-len(cases),1]
ax.set_ylim(ylim)


for i,case in enumerate(cases):
    if case is None:
        continue
    label,me,fla,_,shift=sgms[case][:5]
    sty=fla2sty[fla]
    
    if doShift:
        if len(me)==2:
            me=(me[0]+shift[0],me[1],shift[1])
        elif len(me)==3:
            me=(me[0]+shift[0],me[1],np.sqrt(me[2]**2+shift[1]**2))
    
    if len(me)==2:
        ax.errorbar(x=me[0] ,xerr=me[1],y=[-i], color=sty[0],fmt=sty[1], capsize=5,markersize=5.5)
    elif len(me)==3:
        errTot=np.sqrt(me[1]**2+me[2]**2)
        if doShift:
            print(case,me,errTot)
        ax.errorbar(x=me[0] ,xerr=me[1],y=[-i], color=sty[0],fmt=sty[1], capsize=5,markersize=5.5)
        ax.errorbar(x=me[0] ,xerr=errTot,y=[-i], color=sty[0],fmt=sty[1], capsize=5,markersize=5.5)
    else:
        1/0
    if case=='this':
        mean=me[0]; err=me[1] if len(me)==2 else np.sqrt(me[1]**2+me[2]**2)
        ax.fill_betweenx(y=ylim, x1=mean-err, x2=mean+err, alpha=0.2, color='red')

print()

ax.yaxis.set_ticks([-i for i in range(len(cases))])
ax.set_yticklabels([sgms[case][0] if case is not None else '' for case in cases],fontsize=12)

ax.text(72,1-1,r'with $\mathcal{J}_{N\sigma}$',fontsize=15,va='center')
for place,text in zip([0,4,6,9],['standard','with prior','F-H method','pheno.']):
    ax.fill_between(x=xlim, y1=-place-0.5-1e-5,y2=-place-0.5+1e-5, color='grey')
    ax.text(72,-place-1,text,fontsize=15,va='center')

plt.tight_layout()
plt.savefig(f'fig/sgmCompare.pdf' if not doShift else f'fig/sgmCompare_shift.pdf')
plt.close()

None

def subtract(me1,me2):
    return (me1[0]-me2[0],np.sqrt(me1[1]**2+me2[1]**2))

t=subtract(sgms['this'][1],sgms['this2'][1])
print(yu.un2str(t[0],t[1]))
t=subtract(sgms['Mainz23_prior'][1],sgms['Mainz23_wd'][1])
print(yu.un2str(t[0],t[1]))
t=subtract(sgms['PNDME21_prior'][1],sgms['PNDME21'][1])
print(yu.un2str(t[0],t[1]))
t=subtract(sgms['PNDME25_prior'][1],sgms['PNDME25'][1])
print(yu.un2str(t[0],t[1]))

print()
t=subtract(sgms['Hofe23'][1],sgms['this'][1])
print(yu.un2str(t[0],t[1]),t[0]/t[1])

print()
t_mN=subtract(sgms['this'][1],(-6.5,2.4))
print(yu.un2str(t_mN[0],t_mN[1]))
t=subtract(sgms['Hofe23'][1],t_mN)
print(yu.un2str(t[0],t[1]),t[0]/t[1])

print()
t_mN_iso=subtract(t_mN,(-3.1,0.5))
print(yu.un2str(t_mN_iso[0],t_mN_iso[1]))
t=subtract(sgms['Hofe23'][1],t_mN_iso)
print(yu.un2str(t[0],t[1]),t[0]/t[1])

print()
t_phe2lat=subtract(sgms['Hofe23'][1],(3.1,0.5))
print(yu.un2str(t_phe2lat[0],t_phe2lat[1]))
t=subtract(t_phe2lat,t_mN)
print(yu.un2str(t[0],t[1]),t[0]/t[1])

this (39.199999999999996, 0.7, 0.5) 0.8602325267042626
this2 (37.3, 1.3, 0.5) 1.392838827718412
ETM24 (38.8, 8.1, 0.5) 8.115417426109392
Mainz23_wd (42.3, 2.4, 0) 2.4
PNDME25 (42, 6, 0) 6.0
Mainz23_prior (46.9, 1.7, 0) 1.7
PNDME25_prior (62, 6, 0) 6.0
RQCD23 (43.9, 4.7, 0) 4.7
BMW20 (37.4, 3.0, 4.1) 5.080354318352215
QCDSF12 (33.9, 8, 6.020797289396148) 10.012492197250394
Hofe23 (55.9, 3.5, 0.5) 3.5355339059327378
Frie19 (53.9, 7, 0.5) 7.0178344238090995
Ruiz18 (54.9, 5, 0.5) 5.024937810560445

1.9(1.5)
4.6(2.9)
17.7(8.9)
20.0(8.5)

22.7(3.6) 6.359766096623969

42.8(2.5)
16.2(4.3) 3.7664234952997453

45.9(2.5)
13.1(4.3) 3.025315410553639

55.9(3.5)
13.1(4.3) 3.025315410553639


In [ ]:
import numpy as np
import math

mpiC=139.57
mpi0=134.98

C=5.9*10**(-5) *10**(3*4)
Fpi=92.2
Z=C/(Fpi**4)

t=math.sqrt(mpi0**2+2*4*np.pi*(1/137) * Fpi**2 * Z)
print('mpiC from EM ChPT:',t)


mpiC=139.57
mpi0=134.98

c1=-0.74 * 10**(-3)
t=-4*c1*(mpiC**2-mpi0**2)
print('Delta sgm @ O(p^2):',t)


mpiC from EM ChPT: 139.61675408003384
Delta sgm @ O(p^2): 3.730146120000009


10.0